# 数当てゲームの実装

実物のプロジェクトに取り組むことで一般的な Rust の概念に触れる（`let`、`match`、メソッド、関連関数、外部クレートの使用などについて基礎的な部分を学ぶ）

ここでは、古典的な初心者向けのプログラミング問題である「数当てゲーム」を実装する

これは以下のように動作する

- プログラムは1から100までの乱数整数を生成する
- プレーヤーに予想を入力するよう促す
- 予想が入力されると、プログラムはその予想が小さすぎたか大きすぎたかを出力する
    - 予想が当たっていれば、祝福メッセージを表示して終了する

## 予想を処理する

数当てプログラムの最初の部分は、ユーザに入力を求め、その入力を処理し、予期した形式になっていることを確認する

まずは、プレーヤーが予想を入力できるようにする

In [2]:
:dep evcxr_input

// => :dep マジックコマンドで evcxr_input 外部クレートをロード
//    evcxr_input: EvCxR Jupyter Kernel 上で標準入力を使えるようにするクレート

In [3]:
// std::io クレートを使う
use std::io;

// 入力を促すメッセージ
println!("数当てゲームを始めます！");
// println!("数を予想して入力してください：");

// guess 可変変数を宣言
let mut guess = String::new();

// 標準入力から guess 変数に入力値を格納（※ EvCxR Jupyter 上では使えない）
// io::stdin().read_line(&mut guess).expect("行の読み込みに失敗しました");

// EvCxR Jupyter 上で標準入力から guess 変数に入力値を格納する場合は以下のようにする
guess = evcxr_input::get_string("数を予想して入力してください：");

// 入力値の内容を表示
println!("次のように予想しました：{}", guess);

数当てゲームを始めます！


数を予想して入力してください： 50


次のように予想しました：50


ユーザ入力を受け付け、結果を出力するためには、`io`（入出力）ライブラリをスコープに導入する必要がある

`io` ライブラリは標準ライブラリ（`std`）に存在するため、`use std::io;` という宣言を行うことで使えるようになる

デフォルトでは（`use` 文を使わない状態では）、`std::prelude` に存在するいくつかの型のみが使える

そのため、使用したい型が `prelude` にない場合は、`use` 文で明示的にその型をスコープに導入する必要がある

`std::io` ライブラリを使用することで、ユーザ入力を受け付ける能力などの実用的な機能の多くを使用することができるようになる

ただし、Jupyter 上では `std::io::stdin()` 関数による標準入力は動かないため、ここでは `evcxr_input` 外部クレートの `get_string()` 関数を利用している

したがって上記コードでは、`use std::io;` 文の意味はなくなっている

この場合 `use evcxr_input::get_string;` で evcxr_input クレート内の `get_string` 関数をスコープに導入し、`evcxr_input::get_string` の代わりに `get_string` を使う方が良いかもしれない

In [4]:
// evcxr_input::get_string をスコープに導入
use evcxr_input::get_string;

// EvCxR Input から guess 可変変数に入力値を格納
// use evcxr_input::get_string; により evcxr_input::get_string を get_string だけで呼び出せるようになっている
let mut guess = get_string("数を予想して入力してください：");

println!("次のように予想しました：{}", guess);

数を予想して入力してください： 50


次のように予想しました：50


### println! マクロのプレースホルダーで値を出力する
`println!("次のように予想しました：{}", guess);` の行は、ユーザ入力を保存した文字列の中身を出力している

1組の波括弧（`{}`）は、プレースホルダーの役目を果たす（値を所定の場所に保持する小さなカニのはさみと考える）

最初の波括弧の組は、フォーマット文字列の後に列挙された最初の値に対応し、2組目は2つ目の値...と続いていく

そのため、1回の `println!` の呼び出しで複数の値を出力するコードは、 以下のような形になる

In [5]:
let x = 5;  // 変数 x に 5 を束縛（immutable）
let y = 10; // 変数 y に 10 を束縛（immutable）

println!("x = {}, y = {}", x, y);

x = 5, y = 10


## 秘密の数字を生成する

次に、ユーザが数当てに挑戦する秘密の数字を生成する必要がある

ゲームが何度も楽しめるように、毎回この秘密の数字は変わるべきである

そのため、ゲームが難しくなりすぎないように、1から100までの乱数を使用することにする

Rust の標準ライブラリには乱数機能はまだ含まれていないため、`rand` 外部クレートをインストールして使う

In [6]:
:dep rand = "0.3.14"

### 乱数を生成する
`rand` クレートにある `thread_rng().gen_range(<最小値>, <最大値+1>)` 関数を使うと、指定された `最小値`～`最大値` 内の整数をランダムに生成することが出来る

ここで注意すべきは `gen_range()` 関数の第2引数が `最大値+1` だという点である

そのため、1～100の乱数を取得したい場合は `gen_range(1, 101)` という形で呼び出す必要がある

In [7]:
use rand::Rng;

// secret_number 変数に 1～100 の乱数を束縛
let secret_number = rand::thread_rng().gen_range(1, 101);

println!("秘密の数字：{}", secret_number);

秘密の数字：71


## 予想と秘密の数字を比較する

ユーザ入力と乱数生成ができるようになったため、これらを比較する処理を実装する

ただし、ここで注意しなければならないことは、Rust には **強い静的型システム** があるということである

すなわち、現状ユーザ入力が格納されている `guess` 変数は文字列型（`String`）であるのに対し、秘密の数字が格納されている `secret_number` 変数は（明示されていないが）32ビット整数型（`i32`）であるため、**型の不一致**が発生する

そのため、まずは `guess` 変数を文字列型から整数型に変換する必要がある

In [8]:
// guess 変数を整数型で生成
let guess: i32 = guess.trim().parse()
    .expect("数字を入力してください！"); // 数値型に変換出来なかった場合の例外処理

guess

50

ここで `guess` という名前の変数はすでに文字列型で定義されているにも関わらず、整数型で再度定義し直していることに注目する

Rust では、同一名の変数を新しい値で覆い隠す（shadow）ことが許されている

この機能は **shadowing** と呼ばれ、値を別の型に変換したいシチュエーションでよく使われる

shadowing のおかげで別々の変数を2つ作らされることなく、`guess` という変数名を再利用することができるのである

続いて `guess.trim().parse()` という式を見てみる

この部分における `guess` 変数はまだ文字列型の `guess` 変数である

`String` オブジェクトの `trim` メソッドは、両端の空白をすべて除去する

これはユーザ入力時に、エンターキーによる改行文字が文字列の終端に入ってしまうため、それを除去するために行っている

### Result型で失敗の可能性を扱う
`String` オブジェクトの `parse` メソッドは文字列を数値型に変換するものだが、エラーが発生しやすい

例えば、文字列に数字以外の文字が入っていれば数値に変換できるわけがない

そのため `parse` メソッドは `Result` 型の値を返す

この `Result` 型は、**列挙型**（通常 `enum` 型と呼ばれる）であり、固定された種類の値を持つ

列挙型の持つ固定値は、**列挙子(variant)** と呼ばれる

`Result` 型に関しては、列挙子は `Ok` か `Err` の2つである

`Ok` 列挙子は処理が成功したことを表し、生成された値を中に保持する

`Err` 列挙子は処理が失敗したことを意味し、処理が失敗した過程や失敗理由などの情報を中に保有する

これら `Result` 型の目的は、エラー処理の情報をコード化することである

`Result` 型にも、他の型同様メソッドが定義されている

おそらく最もよく使うメソッドは `expect` メソッドであり、このメソッドは以下のような条件分岐を行う

- `Result` オブジェクトが `Ok` 値の場合は、`Ok` 列挙子が保持する返り値をそのまま返す
    - 今回の場合は文字列を数値型に変換した結果の数値が返る
- `Result` オブジェクトが `Err` 値の場合は、プログラムをクラッシュさせ、 引数として渡された文字列を表示する
    - 今回の場合は「数字を入力してください！」というメッセージを表示して終了する

In [9]:
// std::cmp::Ordering 型をスコープに導入
use std::cmp::Ordering;

// guess 変数と secret_number 変数の値を比較する
match guess.cmp(&secret_number) {
    Ordering::Less => println!("秘密の数字（{}）よりも小さい", secret_number),
    Ordering::Greater => println!("秘密の数字（{}）よりも大きい", secret_number),
    Ordering::Equal => println!("当たり！"),
};

秘密の数字（71）よりも小さい


`cmp` メソッドは2値を比較するメソッドだが、比較できるものに対してなら何に対しても呼び出すことができる

このメソッドは、比較したいものへの参照を取る（**参照**: 値が格納されているメモリ空間のこと; 基本的には変数名の頭に `&` 演算子をつけることで参照をとることができる）

上記では、`guess` 変数と `secret_number` 変数を比較している

その後、`cmp` メソッドは `std::cmp::Ordering` 列挙型の値を返すため、`match` 式を使用して次の動作を決定する

`match` 式は、複数のアームからできているが、一つのアームは以下の要素から構成されている

- パターン
- `match` 式の冒頭で与えた値がパターンにマッチした時に走るコード

Rust は、`match` に与えられた値を取り、各アームのパターンを順番に照合していく

`match` 式とパターンは、コードを書く際に出くわす様々なシチュエーションを表現可能であり、すべてのシチュエーションに対処していることを保証するのを手助けしてくれる

上記で使われている `match` 式で行われている処理の詳細を確認してみる

例えば、 ユーザは `50` と予想し、ランダム生成された秘密の数字が `38` だったとする

コードが 50 と 38 を比較すると、`cmp` メソッドは `Ordering::Greater` 列挙子を返す（50 は 38よりも大きい）

`match` 式に `Ordering::Greater` が与えられ、各アームのパターンを吟味し始める

まず、最初のアームのパターン（`Ordering::Less`）と照合すると、`Ordering::Greater` と `Ordering::Less` はマッチしないため、このアームのコードは無視される

次のアームのパターン（`Ordering::Greater`）は `Ordering::Greater` とマッチするため、このアームに紐づけられたコードが実行され、画面に「秘密の数字（38）よりも大きい」と表示される

この場合、最後のアームと照合する必要はもうないため、これで `match` 式の実行は終わりになる

## ループで複数回の予想を可能にする

ここまでで大方ゲームはうまく動くようになったが、まだユーザは一度しか予想できない

そのため `loop` 式で無限ループを追加し、ユーザが何度も予想できるようにする

In [10]:
// loop {...} で囲まれた処理は何度も実行される
loop {
    // ユーザからの入力を受け付け、文字列型変数 guess に束縛
    let guess: String = evcxr_input::get_string("数を予想して入力してください：");
    
    // guess 変数を文字列型 => 整数型に変換
    // * 数字以外が入力された場合は「数字を入力してください！」と表示してプログラム終了
    let guess: i32 = guess.trim().parse().expect("数字を入力してください！");
    
    // guess 変数と secret_number 変数を比較
    match guess.cmp(&secret_number) {
        Ordering::Less => println!("秘密の数字より小さいです"),
        Ordering::Greater => println!("秘密の数字より大きいです"),
        Ordering::Equal => println!("当たりです！"),
    };
};

数を予想して入力してください： 50


秘密の数字より小さいです


数を予想して入力してください： 75


秘密の数字より大きいです


数を予想して入力してください： 60


...

数を予想して入力してください： 71


当たりです！


数を予想して入力してください： quit


thread '<unnamed>' panicked at '数字を入力してください！: ParseIntError { kind: InvalidDigit }', src/lib.rs:124:43
stack backtrace:
   0: rust_begin_unwind
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/std/src/panicking.rs:493:5
   1: core::panicking::panic_fmt
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/panicking.rs:92:14
   2: core::option::expect_none_failed
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/option.rs:1329:5
   3: run_user_code_9
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Segmentation fault.
   0: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   1: <unknown>
   2: mi_free
   3: alloc::alloc::dealloc
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/alloc/src/alloc.rs:104:14
      <alloc::alloc::Global 

Error: Child process terminated with status: signal: 6

### 正しい予想をした後にゲームを終了する
上記プログラムは、`String::parse` メソッドの習性を利用して、数字以外の文字列を入力すればエラーで終了させることができる

しかし、これはあまりきれいな終了方法とは言えない

本来は、正しい数字が予想された時点でゲームを終了したい

ここで `break` 文を利用すると、ループを正しく抜けることができる

In [11]:
// 全体コードを再掲

use rand::Rng; // rand::thread_rng().gen_range() 関数を使えるようにする
use std::cmp::Ordering; // std::cmp::Ordering 型をスコープに導入

println!("数当てゲームを始めます！");

// 秘密の数字: secret_number 変数に 1～100 の乱数を束縛
let secret_number = rand::thread_rng().gen_range(1, 101);

// 無限ループ処理
loop {
    // ユーザ入力を guess 変数に束縛（文字列型 => 整数型に変換）
    let guess: String = evcxr_input::get_string("数を予想して入力してください：");
    let guess: i32 = guess.trim().parse().expect("数字を入力してください！");
    
    // ユーザ入力値と秘密の数字を比較
    match guess.cmp(&secret_number) {
        Ordering::Less => println!("秘密の数字より小さいです"),
        Ordering::Greater => println!("秘密の数字より大きいです"),
        Ordering::Equal => { // 式は {} で囲むことで複数処理を記述することができる
            println!("当たりです！");
            break; // ループを抜ける
        },
    };
};

数当てゲームを始めます！


数を予想して入力してください： 50


秘密の数字より小さいです


数を予想して入力してください： 75


秘密の数字より小さいです


...

秘密の数字より大きいです


数を予想して入力してください： 82


秘密の数字より大きいです


数を予想して入力してください： 81


当たりです！


## 不正な入力を処理する

さらにゲームの振る舞いを改善するために、ユーザが数値以外を入力した時にプログラムをクラッシュさせるのではなく、非数値を無視してユーザが数当てを続けられるようにする

これは、`guess` 変数が `String` 型から `i32` 型に変換される行を以下のように修正することで達成できる

```rust
// String::parse() メソッドが Result 列挙型を返すことを利用して、列挙子のパターンマッチングを行う
let guess: u32 = match guess.trim().parse() {
    Ok(num) => num, // Ok 列挙子に格納されている値（数値）をそのまま返す
    Err(_) => continue, // 以降の処理をすべてスキップし、ループの最初に戻る
};
```

`expect` メソッドの呼び出しから `match` 式に切り替えることは、エラーでクラッシュする動作からエラー処理を行う処理に変更する一般的な手段になる

`parse` メソッドが、`Ok` か `Err` の列挙子を取りうる `Result` 列挙型を返すことを思い出すと、`match` 式でパターンマッチングできることが分かる

`parse` メソッドは、文字列から数値への変換に成功したら、結果の数値を保持する `Ok` 列挙子を返す

この `Ok` 列挙子は、最初のアームのパターンにマッチし、この `match` 式は `parse` メソッドが生成し `Ok` 値に格納した `num`（変数名は任意）の値を返すだけである

その数値が最終的に、生成している新しい `guess` 変数として束縛されることになる

`parse` メソッドは、文字列から数値への変換に失敗したら、エラーに関する情報を多く含む `Err` 列挙子を返す

この `Err` 列挙子は、最初の `match` アームの `Ok(num)` というパターンにはマッチしないものの、 2番目のアームの `Err(_)` というパターンにはマッチする

ここで `_` は、包括値と呼ばれ、ここでは「保持している情報がどんなものでもいいから全ての `Err` 列挙子にマッチさせたい」と宣言している

従って、プログラムは2番目のアームのコードを実行し（`continue` 文）、`loop` の次のステップに移ることになる

## プログラムを完成させる

上記の処理を追加し、さらに Rust のエントリーポイントとなる `main` 関数で囲み、最終的なプログラムを完成させる

In [12]:
/// 数当てゲーム

use rand::Rng; // rand::thread_rng().gen_range() 関数を使えるようにする
use std::cmp::Ordering; // std::cmp::Ordering 型をスコープに導入

// Rust プログラムエントリーポイント
fn main() {
    println!("数当てゲームを始めます！");
    
    // 秘密の数字: secret_number 変数に 1～100 の乱数を束縛
    let secret_number = rand::thread_rng().gen_range(1, 101);

    // 無限ループ処理
    loop {
        // ユーザ入力を guess 変数に束縛（文字列型 => 整数型に変換）
        let guess: String = evcxr_input::get_string("数を予想して入力してください：");
        let guess: u32 = match guess.trim().parse() {
            Ok(num) => num, // Ok 列挙子に格納されている値（数値）をそのまま返す
            Err(_) => continue, // 以降の処理をすべてスキップし、ループの最初に戻る
        };

        // ユーザ入力値と秘密の数字を比較
        match guess.cmp(&secret_number) {
            Ordering::Less => println!("秘密の数字より小さいです"),
            Ordering::Greater => println!("秘密の数字より大きいです"),
            Ordering::Equal => { // 式は {} で囲むことで複数処理を記述することができる
                println!("当たりです！");
                break; // ループを抜ける
            },
        };
    };
}

In [13]:
// REPL 環境では main 関数を明示的に呼び出す
main();

数当てゲームを始めます！


数を予想して入力してください： quit
数を予想して入力してください： 50


秘密の数字より大きいです


数を予想して入力してください： 25


秘密の数字より大きいです


...

秘密の数字より小さいです


数を予想して入力してください： 3


秘密の数字より小さいです


数を予想して入力してください： 4


当たりです！
